In [166]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, widget, Layout, Button, Box, FloatText, Textarea, Dropdown, Label, VBox
from IPython.display import display
from IPython.html import widgets
from prettytable import PrettyTable
from serving_client import ServingClient
from game_client import Game_Client

form_item_layout = Layout(
    display='inline-flex',
    flex_flow='row',
    flex_wrap= 'wrap',
    justify_content='space-between',
    max_width='0.8'
)

#Creating required variable
prevgameid = None
gameid = widgets.Text()
GAME_ID_FLAG = 'GAME_ID_FLAG'
update_data = Button(description='Update Data', button_style='danger')
model = widgets.Text(value='6-lgbm')
version = widgets.Text(value = '1.0.1')
workspace= widgets.Text(value='axelbogos')
update_model = Button(description='Update Model', button_style='danger')
out = widgets.Output()

game_data = None

serving_client = ServingClient()
game_client = None

def update_game_ID(gameid=gameid.value, prevgameid=prevgameid, game_data=game_data):
    if gameid != prevgameid:
        game_client = Game_Client(game_id = int(gameid.value))
        game_data = game_client.ping_api()
        if game_data != None:
            if game_data.shape[0] > 0:
                predictions = serving_client.predict(game_data)
                game_data['predict'] = predictions
    else:
        game_data_temp = game_client.ping_api()
        if game_data_temp != None:
            if game_data_temp.shape[0] > 0:
                predictions = serving_client.predict(game_data_temp)
                game_data_temp['predict'] = predictions
                game_data.append(game_data_temp)
    return game_data

def create_dashboard(game_data=game_data):
    home_team = game_data.iloc[0]["home_team"]
    away_team = game_data.iloc[0]["away_team"]
    
    x = PrettyTable(["Home Team", "Away Team"])
    x.add_row([home_team, away_team])
    x.add_row(game_data.predict[game_data["team"]== game_data["home_team"]].sum(),
                                game_data.predict[game_data["team"]== game_data["away_team"]].sum())
    return x        

def on_game_button_click(b):
    game_data = update_game_ID()
    if game_data != None:
        if game_data.shape[0] > 0:
            x=create_dashboard(game_data)
            
            with out:
                    out.clear_output()
                    display(print(x.get_string(title=f"Period {game_client.period} - {game_client.time_left}")))
                    display(game_data)
    else:
        with out:
            out.clear_output()
            display('No data yet for this game')



def on_model_button_click(b):
    serving_client.download_registry_model(workspace.value,model.value,version.value)
    if game_data != None:
        if game_data.shape[0] > 0:
            predictions = serving_client.predict(game_data)
            game_data['predict'] = predictions
            x=create_dashboard(game_data)
    with out:
            out.clear_output()
            display('model updated')
            if game_data != None:
                if game_data.shape[0] > 0:
                    display(print(x.get_string(title=f"Period {game_client.period} - {game_client.time_left}")))
                    display(game_data)
            else:
                display('No data yet for this game')

        

update_data.on_click(on_game_button_click)
update_model.on_click(on_model_button_click)

box1 = HBox([Label(value='Workspace',layout=Layout(display="flex")),
      workspace, Label(value='Model:'), model], layout=form_item_layout)
box2 =HBox([Label(value='version'), version, update_model], layout=form_item_layout)
box3 = HBox([Label(value='GameID'),
        gameid,update_data], layout=form_item_layout)
box4 = HBox([out],layout=form_item_layout)

VBox([box1,box2,box3,box4])

list available model with corresponding version